In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import sys
sys.path.append('..')
from PIL import Image
from tqdm import trange
import gymnasium as gym
import numpy as np
from sapien.core import Pose
import matplotlib.pyplot as plt
import os
import warnings
import shutil
import random
import math
from plot_utils import *
from itertools import product, combinations, permutations
#!wget https://dl.fbaipublicfiles.com/habitat/ReplicaCAD/hab2_bench_assets.zip -P data
#!cd data && unzip -q hab2_bench_assets.zip -d hab2_bench_assets
from utils import *
from agent_env_utils import *
from config import (
    data_dir, initial_dir, final_dir,
    size_types, space_types, bound_types,
    moving_directions, color_maps,
    potential_sizes, direction_length, scales,
    background_quadrants, directions
)
from scene_helpers import get_position_range, create_config, create_directories, finalize_quadrant

In [2]:
def place_main_objects(env, configs):
    base_size = 0.08
    main_placements = [(i + 1, i, "right") for i in range(len(configs) - 1)]
    distance = np.random.uniform(3 * base_size, 3.5 * base_size) * np.sqrt(2)
    
    for direction_config in main_placements:
        move_obj, ref_obj, direction = direction_config
        if direction == "top" and configs[move_obj]['obj_type'] == "custom":
            new_distance = distance + 2 * base_size 
        else:
            new_distance = distance
        env.spawn_next(direction_config, distance=new_distance)

        # adjust the z position for main obj
        z_offset = configs[move_obj]['size'] * 0.5  
        current_pose = env.objects[move_obj].get_pose()  
        new_z_position = current_pose.p[2] + z_offset
        env.objects[move_obj].set_pose(Pose([current_pose.p[0], current_pose.p[1], new_z_position]) )


def place_bound_objects(env, configs, bounds, bound_directions):
    for k, bound_id in enumerate(bounds):
        id_offset = k * 7 + len(configs)
        
        if bound_directions[k] == "front":
            bound_direction_configs = [
        (0+id_offset, bound_id, "left"), (bound_id, bound_id, "right"),
        (1+id_offset, 0+id_offset, "behind"), (2+id_offset, 1+id_offset, "right"),
        (3+id_offset, 2+id_offset, "right"), (4+id_offset, 0+id_offset, "front"),
        (5+id_offset, 4+id_offset, "right"), (6+id_offset, 5+id_offset, "right")
    ]
            distances = [2 * configs[bound_id]['size']] * len(bound_direction_configs)
            env.place_cubes_in_direction(bound_direction_configs, distances=distances)

        elif bound_directions[k] == "top":
            bound_direction_configs = [
                (0 + id_offset, bound_id, "left"), (1 + id_offset, bound_id, "right"),
                (2 + id_offset, 0 + id_offset, "top"), (3 + id_offset, 1 + id_offset, "top"),
                (4 + id_offset, 2 + id_offset, "top"), (5 + id_offset, 3 + id_offset, "top"),
                (6 + id_offset, 5 + id_offset, "left")
            ]
            distances = [2.5 * configs[bound_id]['size'] if cfg[2] == "top" else 2 * configs[bound_id]['size'] 
                         for cfg in bound_direction_configs]
        env.place_cubes_in_direction(bound_direction_configs, distances=distances)
   

In [11]:
# Define directories
data_dir = "../benchmark/spatial/spatial_occupancy"
final_dir = create_directories(data_dir, empty=True)

# Initialize pair of colors
color_keys = color_maps.keys()
color_pairs = list(itertools.permutations(color_keys, 2))
random.seed(42)
random.shuffle(color_pairs)

# Define ranges and object parameters
obj_type = ['cube', 'cube-bound','sphere', 'sphere-bound']

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    env = gym.make("CustomEnv-v0", obs_mode="rgbd")

    outputs = []
    idx = 0
    for background in trange(4):
        for quadrant_idx, quadrant in enumerate(background_quadrants[background]):
            # Decide the boundary of each background
            object_position_range= get_position_range(background_quadrants, background, scale=0.6)
            ranges = [tuple(range) for range in object_position_range]
            for size_settings in itertools.permutations(size_types, len(size_types)):
                for obj_names in list(itertools.permutations(obj_type, 2)):
                    for colors in color_pairs:
                        colors = [color_maps[color] for color in colors]
                        num_bound_cases = sum(1 for obj in obj_names if "bound" in obj)
                        for bound_directions in list(itertools.product(bound_types, repeat=num_bound_cases)):
                            static =  True
                            configs = []
                            bounds = []
                            
                            #create main object configurations
                            for i, obj_name in enumerate(obj_names):
                                obj_type, obj_id = (obj_name.split("-") + [None])[:2]
                                size = size_settings[i] * 0.02 + np.random.uniform(0, 0.01)
                                actual_size = size * 1.1 / 2 if obj_type == "custom" else size
                                config = create_config(obj_type, colors[i], actual_size, ranges, static or "top" in bound_directions)
                                configs.append(config)
    
                                # Create configurations for bound objects
                                if obj_id == "bound":
                                    bounds.append(i)      
                            bound_configs = [configs[bound_id].copy() for bound_id in bounds for _ in range(7)]
                            
                            # register configures 
                            env.register_configures(collate_infos(configs))
                            if bound_configs:
                                env.register_configures(collate_infos(bound_configs), replace=False)                   
                            _ = env.reset(options={"reconfigure": True})
                            env.initialize_objects(regular=True, background=background)
                            
                            #place items
                            place_main_objects(env, configs)                          
                            place_bound_objects(env, configs, bounds, bound_directions)

                            save_dir = f"{final_dir}/episode{idx}.png"
                            views = [f"front{background}", f"side{background}", f"top{background}"]
                            collect_and_save(env, save_dir, steps=1, mode=views)
           
                            output = {
                                'episode_id': idx,
                                "source": save_dir,
                                "obj_names": obj_names,
                                'sizes': [config['size'] for config in configs],
                                 "colors": [config['color'] for config in configs],
                                 "initiate_area": ranges,
                                 "bound_directions":list(bound_directions),
                                 "quadrant": quadrant_idx,
                                 "background": background,
                            }
                            outputs.append(output)
                            idx += 1
                            
    
    write_out(f"{data_dir}/index.jsonl", outputs)

del env

print("Spatial Occupancy data generation complete.")

[2024-10-15 06:19:06.161] [svulkan2] [warning] A second renderer will share the same internal context with the first one. Arguments passed to constructor will be ignored.
  0%|          | 0/4 [00:10<?, ?it/s]


KeyboardInterrupt: 

In [8]:
write_out(f"{data_dir}/index.jsonl", outputs)